# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""../../../../da…",1.2927e19,"""82299586105982…","""16345251099661…","""17320931994270…","""13677497111334…",false
"""../../../../da…",1.4856e19,"""91445703200084…","""84161993733598…","""17320931994270…","""11772241724612…",true
"""../../../../da…",5.6458e17,"""64923787667438…","""17495943974455…","""16221631072643…","""13677497111334…",false
"""../../../../da…",4.0723e18,"""12009714254368…","""84161993733598…","""53498977082632…","""13677497111334…",false
"""../../../../da…",1.4677e19,"""16509679606296…","""16345251099661…","""53498977082632…","""78498363096648…",true
"""../../../../da…",1.6330e19,"""70677712132910…","""17495943974455…","""14496526988389…",null,false
"""../../../../da…",8.7675e18,"""12101791327672…","""17495943974455…","""53498977082632…","""78498363096648…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",1.2927e19,"""69821814448024…","""16069124074893…","""FirstMortgage3…","""16590765587857…","""Rejected"""
"""../../../../da…",1.4856e19,"""21150438423391…","""94927289039809…","""FirstMortgage3…","""12152318940070…","""Accepted"""
"""../../../../da…",5.6458e17,"""86954258943863…","""10929142510653…","""MoneyMarketSav…","""16590765587857…","""Rejected"""
"""../../../../da…",4.0723e18,"""56644418710798…","""94927289039809…","""BasicChecking""","""16590765587857…","""Rejected"""
"""../../../../da…",1.4677e19,"""17780968865749…","""16069124074893…","""BasicChecking""","""16460894136607…","""Accepted"""
"""../../../../da…",1.6330e19,"""16082519737559…","""10929142510653…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",8.7675e18,"""12579112419015…","""10929142510653…","""BasicChecking""","""16460894136607…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",1.2927e19,"""11783265533226…","""56022105355865…","""FirstMortgage3…","""97309701193113…","""Rejected"""
"""../../../../da…",1.4856e19,"""13948911682568…","""11164975078659…","""FirstMortgage3…","""14162014216966…","""Accepted"""
"""../../../../da…",5.6458e17,"""90673619185066…","""37186626038496…","""MoneyMarketSav…","""97309701193113…","""Rejected"""
"""../../../../da…",4.0723e18,"""58899022123778…","""11164975078659…","""BasicChecking""","""97309701193113…","""Rejected"""
"""../../../../da…",1.4677e19,"""12210370061757…","""56022105355865…","""BasicChecking""","""59153038419005…","""Accepted"""
"""../../../../da…",1.6330e19,"""18677580829725…","""37186626038496…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",8.7675e18,"""18119638784188…","""37186626038496…","""BasicChecking""","""59153038419005…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_CLV=PREDICTOR_1
Customer_City=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""29997154804696…","""17691665586401…","""FirstMortgage3…","""14805960092797…","""Rejected"""
1.4856e19,"""15280917167353…","""39750281202880…","""FirstMortgage3…","""10843939289582…","""Accepted"""
5.6458e17,"""97259935465254…","""14943149091940…","""MoneyMarketSav…","""14805960092797…","""Rejected"""
4.0723e18,"""62056626705874…","""39750281202880…","""BasicChecking""","""14805960092797…","""Rejected"""
1.4677e19,"""66880383314006…","""17691665586401…","""BasicChecking""","""38931764541874…","""Accepted"""
1.6330e19,"""15402556903725…","""14943149091940…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""61745558259341…","""14943149091940…","""BasicChecking""","""38931764541874…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'